In [1]:
import pandas as pd
from NelsonCheck import NelsonCheck as NC
import numpy as np
import os
import cv2
import sklearn
from sklearn.model_selection import train_test_split

import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential

from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten 
from keras.layers import Conv2D, MaxPooling2D 
from keras import backend as K 
import cv2

In [2]:
tagging_df = pd.read_csv("cavite_outbreak.csv", parse_dates=['date'])
tagging_df

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_24772\2674946331.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  tagging_df = pd.read_csv("cavite_outbreak.csv", parse_dates=['date'])


,loc,cases,deaths,date,outbreak
0,CAVITE,64,0,2016-10-01,0
1,CAVITE,214,1,2016-01-17,0
2,CAVITE,161,2,2016-01-24,0
3,CAVITE,147,1,2016-01-31,0
4,CAVITE,160,0,2016-07-02,0
...,...,...,...,...,...
256,CAVITE,60,0,2020-12-13,0
257,CAVITE,60,0,2020-12-20,0
258,CAVITE,57,0,2020-12-27,0
259,CAVITE,68,0,2021-03-01,0


In [3]:
loc_df = tagging_df[tagging_df['loc'] == "CAVITE"]

In [4]:
train = loc_df[loc_df['date'].dt.year < 2019 ]
test = loc_df[loc_df['date'].dt.year == 2019]

In [5]:
train.shape, test.shape, \
"outbreak_train: " + str(train['outbreak'].sum()) + "/" + str(len(train.index)), \
"outbreak_test: " + str(test['outbreak'].sum()) + "/" + str(len(test.index))


((156, 5), (51, 5), 'outbreak_train: 74/156', 'outbreak_test: 29/51')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(train[['cases','deaths','date']], train['outbreak'], test_size=0.33, random_state=42)

In [13]:
x_train_arr = []
x_test_arr = []
city = "CAVITE"
layer = "ndvi"

for date in X_train.date.values:
    full_date_txt = np.datetime_as_string(date, unit='D')
    data_path = city +  "/" + layer + "/snapshot-" + full_date_txt + "T00_00_00Z.jpg"
    print(data_path)
    img = cv2.imread(data_path)
    img = cv2.resize(img, (224,224))
    x_train_arr.append(img)

for date in X_test.date.values:
    full_date_txt = np.datetime_as_string(date, unit='D')
    data_path = city +  "/" + layer + "/snapshot-" + full_date_txt + "T00_00_00Z.jpg"
    img = cv2.imread(data_path)
    img = cv2.resize(img, (224,224))
    x_test_arr.append(img)

CAVITE/ndvi/snapshot-2016-07-17T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2018-04-15T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2018-08-04T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2016-10-23T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2016-07-02T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2016-08-21T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2017-08-27T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2018-05-13T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2017-09-04T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2016-01-05T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2018-11-11T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2016-03-20T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2018-02-18T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2018-11-18T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2016-10-01T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2018-02-25T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2017-02-04T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2016-11-13T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2017-07-30T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2016-07-24T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2016-10-16T00_00_00Z.jpg
CAVITE/ndvi/snapshot-2018-01-14T00

In [14]:
train_x=np.array(x_train_arr)
test_x=np.array(x_test_arr)

train_x=train_x/255.0
test_x=test_x/255.0

In [15]:
# Target categorical Variable

y_train = keras.utils.to_categorical(y_train, 10) 
y_test = keras.utils.to_categorical(y_test, 10)

In [16]:
img_rows, img_cols = 224, 224 
input_shape = (img_rows, img_cols, 3)

model = Sequential() 
model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape)) 
model.add(Conv2D(64, (3, 3), activation = 'relu')) 
model.add(MaxPooling2D(pool_size = (2, 2))) 
model.add(Dropout(0.25))
model.add(Flatten()) 
model.add(Dense(128, activation = 'relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(10, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 flatten (Flatten)           (None, 774400)            0         
                                                                 
 dense (Dense)               (None, 128)               99123328  
                                                        

In [17]:
# Compile
# loss = 'sparse_categorical_crossentropy', optimizer='adam'
model.compile(loss = 'categorical_crossentropy', optimizer = 'adadelta', metrics = ['accuracy'])

In [21]:
# Fit model
model.fit(train_x, y_train, batch_size = 128, epochs = 200, verbose = 2, validation_data = (test_x, y_test))

Epoch 1/200
1/1 - 4s - loss: 1.1710 - accuracy: 0.5288 - val_loss: 1.0606 - val_accuracy: 0.4615 - 4s/epoch - 4s/step
Epoch 2/200
1/1 - 4s - loss: 1.1516 - accuracy: 0.5000 - val_loss: 1.0409 - val_accuracy: 0.4615 - 4s/epoch - 4s/step
Epoch 3/200
1/1 - 7s - loss: 1.1140 - accuracy: 0.5481 - val_loss: 1.0026 - val_accuracy: 0.4615 - 7s/epoch - 7s/step
Epoch 4/200
1/1 - 8s - loss: 1.1136 - accuracy: 0.4808 - val_loss: 0.9881 - val_accuracy: 0.4615 - 8s/epoch - 8s/step
Epoch 5/200
1/1 - 8s - loss: 1.0497 - accuracy: 0.6250 - val_loss: 0.9714 - val_accuracy: 0.4615 - 8s/epoch - 8s/step
Epoch 6/200
1/1 - 8s - loss: 1.0794 - accuracy: 0.5000 - val_loss: 0.9698 - val_accuracy: 0.4615 - 8s/epoch - 8s/step
Epoch 7/200
1/1 - 7s - loss: 0.9624 - accuracy: 0.5865 - val_loss: 0.9315 - val_accuracy: 0.4615 - 7s/epoch - 7s/step
Epoch 8/200
1/1 - 7s - loss: 1.0376 - accuracy: 0.4904 - val_loss: 0.9358 - val_accuracy: 0.4615 - 7s/epoch - 7s/step
Epoch 9/200
1/1 - 7s - loss: 0.9672 - accuracy: 0.6058 -

Epoch 70/200
1/1 - 7s - loss: 0.7701 - accuracy: 0.5962 - val_loss: 0.7303 - val_accuracy: 0.4615 - 7s/epoch - 7s/step
Epoch 71/200
1/1 - 7s - loss: 0.8029 - accuracy: 0.5577 - val_loss: 0.7379 - val_accuracy: 0.4615 - 7s/epoch - 7s/step
Epoch 72/200
1/1 - 6s - loss: 0.8085 - accuracy: 0.5577 - val_loss: 0.7296 - val_accuracy: 0.4615 - 6s/epoch - 6s/step
Epoch 73/200
1/1 - 6s - loss: 0.7916 - accuracy: 0.5769 - val_loss: 0.7578 - val_accuracy: 0.4615 - 6s/epoch - 6s/step
Epoch 74/200
1/1 - 7s - loss: 0.6833 - accuracy: 0.7115 - val_loss: 0.7278 - val_accuracy: 0.4615 - 7s/epoch - 7s/step
Epoch 75/200
1/1 - 7s - loss: 0.8602 - accuracy: 0.5192 - val_loss: 0.7370 - val_accuracy: 0.4615 - 7s/epoch - 7s/step
Epoch 76/200
1/1 - 6s - loss: 0.7229 - accuracy: 0.6538 - val_loss: 0.7343 - val_accuracy: 0.4615 - 6s/epoch - 6s/step
Epoch 77/200
1/1 - 7s - loss: 0.7335 - accuracy: 0.6731 - val_loss: 0.7217 - val_accuracy: 0.4615 - 7s/epoch - 7s/step
Epoch 78/200
1/1 - 6s - loss: 0.7413 - accuracy:

Epoch 139/200
1/1 - 4s - loss: 0.6633 - accuracy: 0.6923 - val_loss: 0.6859 - val_accuracy: 0.5385 - 4s/epoch - 4s/step
Epoch 140/200
1/1 - 4s - loss: 0.6769 - accuracy: 0.7019 - val_loss: 0.6650 - val_accuracy: 0.6154 - 4s/epoch - 4s/step
Epoch 141/200
1/1 - 4s - loss: 0.6607 - accuracy: 0.6635 - val_loss: 0.6830 - val_accuracy: 0.5385 - 4s/epoch - 4s/step
Epoch 142/200
1/1 - 4s - loss: 0.6681 - accuracy: 0.6346 - val_loss: 0.7242 - val_accuracy: 0.4615 - 4s/epoch - 4s/step
Epoch 143/200
1/1 - 4s - loss: 0.6417 - accuracy: 0.6923 - val_loss: 0.7040 - val_accuracy: 0.5385 - 4s/epoch - 4s/step
Epoch 144/200
1/1 - 4s - loss: 0.6524 - accuracy: 0.6923 - val_loss: 0.7083 - val_accuracy: 0.5385 - 4s/epoch - 4s/step
Epoch 145/200
1/1 - 4s - loss: 0.6409 - accuracy: 0.7019 - val_loss: 0.7103 - val_accuracy: 0.5385 - 4s/epoch - 4s/step
Epoch 146/200
1/1 - 4s - loss: 0.6644 - accuracy: 0.6731 - val_loss: 0.6919 - val_accuracy: 0.5385 - 4s/epoch - 4s/step
Epoch 147/200
1/1 - 4s - loss: 0.6209 - 

In [22]:
# Evaluate Model
score = model.evaluate(test_x, y_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.6713906526565552
Test accuracy: 0.5384615659713745


In [23]:
# Predictions
pred = model.predict(test_x) 
pred = np.argmax(pred, axis = 1)[:5] 
label = np.argmax(y_test,axis = 1)[:5] 

print(pred) 
print(label)

2/2 [==============================] - 0s 166ms/step
[0 0 0 0 0]
[1 0 1 1 0]
